# Day14_0: 군집화 (Clustering)

## 📚 학습 목표

**Part 1: 기초 - K-Means**
1. 군집화(Clustering)의 개념과 지도학습과의 차이 이해하기
2. K-Means 알고리즘의 원리와 구현 방법 익히기
3. Elbow Method로 최적 군집 수 찾기
4. Silhouette Score로 군집 품질 평가하기
5. px.scatter()로 군집 결과 시각화하기

**Part 2: 심화 - 다양한 군집화 알고리즘**
1. 계층적 군집화(Hierarchical Clustering)와 Dendrogram 해석하기
2. DBSCAN으로 밀도 기반 군집화 및 노이즈 탐지하기
3. GMM(Gaussian Mixture Model)으로 확률적 군집화 수행하기
4. 고객 세분화(Customer Segmentation) 실무 프로젝트 구현하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| K-Means | 빠른 군집화, 대용량 처리 | 고객 세분화, 이미지 압축 |
| Elbow/Silhouette | 최적 군집 수 결정 | 마케팅 타겟 그룹 수 결정 |
| 계층적 군집화 | 계층 구조 파악 | 상품 카테고리, 조직 분류 |
| DBSCAN | 이상치 탐지, 임의 모양 | 사기 탐지, 지리 클러스터 |
| GMM | 확률 기반 소속 | 소프트 클러스터링, 혼합 분포 |

**분석가 관점**: 군집화는 비지도학습의 핵심입니다. 레이블 없이 데이터의 숨겨진 패턴을 발견하여 고객 세분화, 추천 시스템, 이상 탐지 등 다양한 비즈니스 문제를 해결합니다!

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# sklearn 군집화 알고리즘
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

# sklearn 유틸리티
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import make_blobs, make_moons

# 계층적 군집화 시각화
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
np.random.seed(42)

print("환경 설정 완료!")

## 데이터셋 준비: 고객 세분화용 데이터

Mall Customer Segmentation 스타일의 고객 데이터를 생성합니다.

In [ ]:
# 고객 세분화용 합성 데이터 생성
np.random.seed(42)
n_customers = 500

# 4개의 고객 그룹 생성
# 그룹 1: 저소득-저소비 (예산 의식형)
group1 = np.random.multivariate_normal([30, 20], [[50, 10], [10, 100]], 125)
# 그룹 2: 저소득-고소비 (충동 소비형)
group2 = np.random.multivariate_normal([30, 80], [[50, 10], [10, 100]], 125)
# 그룹 3: 고소득-저소비 (절약형)
group3 = np.random.multivariate_normal([80, 20], [[50, 10], [10, 100]], 125)
# 그룹 4: 고소득-고소비 (프리미엄형)
group4 = np.random.multivariate_normal([80, 80], [[50, 10], [10, 100]], 125)

# 데이터 합치기
customer_data = np.vstack([group1, group2, group3, group4])

# 값 범위 조정 (0-100)
customer_data = np.clip(customer_data, 1, 100)

# DataFrame 생성
df_customers = pd.DataFrame({
    'CustomerID': range(1, n_customers + 1),
    'Annual_Income': customer_data[:, 0].astype(int),  # 연간 소득 (단위: 천만원)
    'Spending_Score': customer_data[:, 1].astype(int)  # 소비 점수 (1-100)
})

# 추가 특성
df_customers['Age'] = np.random.randint(18, 70, n_customers)
df_customers['Visit_Frequency'] = np.random.randint(1, 30, n_customers)  # 월 방문 횟수

print("고객 데이터 크기:", df_customers.shape)
print("\n컬럼 설명:")
print("- CustomerID: 고객 ID")
print("- Annual_Income: 연간 소득 (1-100, 단위: 천만원)")
print("- Spending_Score: 소비 점수 (1-100, 쇼핑몰 소비 활동 점수)")
print("- Age: 나이")
print("- Visit_Frequency: 월 방문 횟수")

df_customers.head(10)

In [ ]:
# 기본 통계
df_customers.describe()

In [ ]:
# 고객 분포 시각화 (레이블 없이)
fig = px.scatter(df_customers, x='Annual_Income', y='Spending_Score',
                 title='고객 분포: 소득 vs 소비 점수',
                 labels={'Annual_Income': '연간 소득 (천만원)', 
                        'Spending_Score': '소비 점수'},
                 opacity=0.6)
fig.show()

In [ ]:
# 군집화에 사용할 특성 선택 및 스케일링
X = df_customers[['Annual_Income', 'Spending_Score']].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("원본 데이터 범위:")
print(f"  Annual_Income: {X[:, 0].min():.1f} ~ {X[:, 0].max():.1f}")
print(f"  Spending_Score: {X[:, 1].min():.1f} ~ {X[:, 1].max():.1f}")
print("\n스케일링 후 범위:")
print(f"  Annual_Income: {X_scaled[:, 0].min():.2f} ~ {X_scaled[:, 0].max():.2f}")
print(f"  Spending_Score: {X_scaled[:, 1].min():.2f} ~ {X_scaled[:, 1].max():.2f}")

---

# Part 1: 기초 - K-Means

---

## 1.1 군집화(Clustering) 개념

**군집화**는 비지도학습(Unsupervised Learning)의 대표적인 기법으로, 레이블 없이 데이터를 유사한 그룹으로 묶습니다.

| 지도학습 | 비지도학습 (군집화) |
|---------|-------------------|
| 레이블 있음 | 레이블 없음 |
| 정답을 예측 | 패턴을 발견 |
| 분류, 회귀 | 군집화, 차원축소 |
| 정확도로 평가 | Silhouette 등으로 평가 |

---

## 1.2 K-Means 알고리즘

K-Means는 가장 널리 사용되는 군집화 알고리즘입니다.

**알고리즘 단계**:
1. K개의 중심점(centroid) 무작위 초기화
2. 각 데이터 포인트를 가장 가까운 중심점에 할당
3. 각 군집의 중심점 재계산
4. 2-3 반복 (수렴할 때까지)

**장점**: 빠름, 구현 간단, 대용량 처리  
**단점**: K를 미리 지정, 구형 군집 가정, 초기값에 민감

In [ ]:
# K-Means 기본 사용법 (K=4로 시작)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans.fit(X_scaled)

# 군집 레이블과 중심점
labels = kmeans.labels_
centers = kmeans.cluster_centers_

print("군집 레이블 (처음 10개):", labels[:10])
print("\n각 군집의 데이터 수:")
for i in range(4):
    print(f"  군집 {i}: {(labels == i).sum()}개")
print(f"\nInertia (군집 내 거리 합): {kmeans.inertia_:.2f}")

In [ ]:
# 군집 결과 시각화 (px.scatter)
df_result = df_customers.copy()
df_result['Cluster'] = labels.astype(str)

# 중심점을 원래 스케일로 변환
centers_original = scaler.inverse_transform(centers)

fig = px.scatter(df_result, x='Annual_Income', y='Spending_Score',
                 color='Cluster', 
                 title='K-Means 군집화 결과 (K=4)',
                 labels={'Annual_Income': '연간 소득 (천만원)', 
                        'Spending_Score': '소비 점수'},
                 color_discrete_sequence=px.colors.qualitative.Set1,
                 opacity=0.7)

# 중심점 추가
fig.add_trace(go.Scatter(
    x=centers_original[:, 0], y=centers_original[:, 1],
    mode='markers', marker=dict(symbol='x', size=15, color='black', line=dict(width=2)),
    name='중심점 (Centroid)'
))

fig.show()

### 💡 실무 예시: 군집 해석

각 군집의 특성을 분석하여 비즈니스 인사이트를 도출합니다.

In [ ]:
# 군집별 특성 분석
df_result['Cluster_Num'] = labels

cluster_summary = df_result.groupby('Cluster_Num').agg({
    'Annual_Income': 'mean',
    'Spending_Score': 'mean',
    'Age': 'mean',
    'CustomerID': 'count'
}).round(1)

cluster_summary.columns = ['평균 소득', '평균 소비점수', '평균 나이', '고객 수']

# 군집 이름 부여
cluster_names = []
for idx, row in cluster_summary.iterrows():
    income = '고소득' if row['평균 소득'] > 50 else '저소득'
    spending = '고소비' if row['평균 소비점수'] > 50 else '저소비'
    cluster_names.append(f"{income}-{spending}")

cluster_summary['군집 특성'] = cluster_names
print("=== 군집별 특성 요약 ===")
cluster_summary

---

## 1.3 Elbow Method로 최적 K 찾기

K 값에 따른 Inertia(군집 내 거리 합) 변화를 관찰하여 **팔꿈치(elbow)** 지점을 찾습니다.

- Inertia가 급격히 감소하다가 완만해지는 지점이 최적 K

In [ ]:
# Elbow Method
K_range = range(1, 11)
inertias = []

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_scaled)
    inertias.append(kmeans_temp.inertia_)

# Elbow Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(K_range), y=inertias, mode='lines+markers',
                         marker=dict(size=10), line=dict(width=2)))

# Elbow 지점 표시 (K=4)
fig.add_vline(x=4, line_dash="dash", line_color="red", 
              annotation_text="Elbow (K=4)")

fig.update_layout(title='Elbow Method: 최적 군집 수 찾기',
                  xaxis_title='군집 수 (K)',
                  yaxis_title='Inertia (군집 내 거리 합)')
fig.show()

### 💡 실무 팁: Elbow 해석

- K=4에서 급격한 감소가 완만해지는 "팔꿈치" 형태
- K를 늘려도 Inertia 감소 효과가 미미해지는 지점이 최적

---

## 1.4 Silhouette Score로 군집 품질 평가

Silhouette Score는 군집화 품질을 -1 ~ 1 범위로 평가합니다.

$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

- **a(i)**: 같은 군집 내 평균 거리 (응집도)
- **b(i)**: 가장 가까운 다른 군집과의 평균 거리 (분리도)
- **해석**: 1에 가까울수록 좋음, 0 근처면 경계에 있음, 음수면 잘못 할당

In [ ]:
# K별 Silhouette Score 계산
K_range = range(2, 11)  # K=1은 의미 없음
silhouette_scores = []

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels_temp)
    silhouette_scores.append(score)
    print(f"K={k}: Silhouette Score = {score:.4f}")

print(f"\n최적 K: {list(K_range)[np.argmax(silhouette_scores)]} (Silhouette = {max(silhouette_scores):.4f})")

In [ ]:
# Silhouette Score Plot
fig = go.Figure()
fig.add_trace(go.Bar(x=list(K_range), y=silhouette_scores,
                     marker_color=['red' if s == max(silhouette_scores) else 'steelblue' 
                                   for s in silhouette_scores]))

fig.update_layout(title='Silhouette Score by K (높을수록 좋음)',
                  xaxis_title='군집 수 (K)',
                  yaxis_title='Silhouette Score')
fig.show()

In [ ]:
# 개별 샘플의 Silhouette 값 시각화 (K=4)
kmeans_4 = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_4 = kmeans_4.fit_predict(X_scaled)

# 각 샘플의 Silhouette 값
sample_silhouette = silhouette_samples(X_scaled, labels_4)

# Silhouette Plot 데이터 준비
silhouette_df = pd.DataFrame({
    'Sample': range(len(labels_4)),
    'Cluster': labels_4,
    'Silhouette': sample_silhouette
})

# 클러스터별 정렬
silhouette_df = silhouette_df.sort_values(['Cluster', 'Silhouette'])
silhouette_df['Index'] = range(len(silhouette_df))

fig = px.bar(silhouette_df, x='Silhouette', y='Index', color='Cluster',
             orientation='h', title='Silhouette Plot (K=4)',
             color_discrete_sequence=px.colors.qualitative.Set1)

# 평균선 추가
avg_silhouette = silhouette_score(X_scaled, labels_4)
fig.add_vline(x=avg_silhouette, line_dash="dash", line_color="red",
              annotation_text=f"평균: {avg_silhouette:.3f}")

fig.update_layout(height=600, showlegend=True,
                  yaxis=dict(showticklabels=False))
fig.show()

### 💡 실무 예시: Silhouette 해석

- 모든 군집이 평균선 이상: 좋은 군집화
- 음수 값이 많은 군집: 잘못 할당된 샘플 많음
- 군집 크기가 불균형하면 재검토 필요

---

# Part 2: 심화 - 다양한 군집화 알고리즘

---

## 2.1 계층적 군집화 (Hierarchical Clustering)

계층적 군집화는 데이터 간의 거리를 기반으로 트리 구조(Dendrogram)를 생성합니다.

**Linkage 방법**:
- **single**: 최소 거리 (가장 가까운 점)
- **complete**: 최대 거리 (가장 먼 점)
- **average**: 평균 거리
- **ward**: 분산 최소화 (가장 많이 사용)

In [ ]:
# 계층적 군집화를 위한 샘플 데이터 (전체 데이터는 너무 많음)
np.random.seed(42)
sample_idx = np.random.choice(len(X_scaled), 100, replace=False)
X_sample = X_scaled[sample_idx]

# Linkage 계산
linkage_matrix = linkage(X_sample, method='ward')

print("Linkage Matrix 형태:", linkage_matrix.shape)
print("컬럼: [군집1, 군집2, 거리, 샘플수]")

In [ ]:
# Dendrogram 시각화 (Plotly로 구현)
# scipy dendrogram으로 좌표 계산 후 Plotly로 렌더링
import matplotlib.pyplot as plt

# Dendrogram 좌표 추출
plt.figure(figsize=(12, 6))
dendro = dendrogram(linkage_matrix, truncate_mode='lastp', p=30, 
                    no_plot=False, leaf_rotation=90)
plt.title('Dendrogram (Ward Linkage)')
plt.xlabel('샘플 인덱스')
plt.ylabel('거리')

# 절단선 추가 (4개 군집)
plt.axhline(y=5, color='r', linestyle='--', label='Cut at 4 clusters')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# AgglomerativeClustering 사용
hierarchical = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels_hier = hierarchical.fit_predict(X_scaled)

# 결과 시각화
df_hier = df_customers.copy()
df_hier['Cluster'] = labels_hier.astype(str)

fig = px.scatter(df_hier, x='Annual_Income', y='Spending_Score',
                 color='Cluster',
                 title='계층적 군집화 결과 (Ward Linkage, K=4)',
                 labels={'Annual_Income': '연간 소득', 'Spending_Score': '소비 점수'},
                 color_discrete_sequence=px.colors.qualitative.Set2,
                 opacity=0.7)
fig.show()

print(f"Silhouette Score: {silhouette_score(X_scaled, labels_hier):.4f}")

### 💡 실무 예시: Dendrogram 활용

- **절단 높이 조절**: 원하는 군집 수에 맞게 수평선으로 절단
- **계층 구조 파악**: 어떤 군집이 먼저 합쳐지는지 확인
- **이상치 탐지**: 마지막에 합쳐지는 샘플 = 이상치 가능성

---

## 2.2 DBSCAN (밀도 기반 군집화)

DBSCAN은 밀도가 높은 영역을 군집으로 정의합니다.

**핵심 파라미터**:
- **eps**: 이웃 반경 (epsilon)
- **min_samples**: 핵심점이 되기 위한 최소 이웃 수

**장점**: K 지정 불필요, 임의 모양 군집, 노이즈 탐지  
**단점**: 밀도가 다른 군집 어려움, 파라미터 민감

In [ ]:
# 반달 모양 데이터로 DBSCAN 데모
X_moons, y_moons = make_moons(n_samples=300, noise=0.1, random_state=42)

# K-Means vs DBSCAN 비교
kmeans_moons = KMeans(n_clusters=2, random_state=42)
labels_kmeans_moons = kmeans_moons.fit_predict(X_moons)

dbscan_moons = DBSCAN(eps=0.2, min_samples=5)
labels_dbscan_moons = dbscan_moons.fit_predict(X_moons)

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['K-Means (K=2)', 'DBSCAN'])

# K-Means
for cluster in [0, 1]:
    mask = labels_kmeans_moons == cluster
    fig.add_trace(go.Scatter(x=X_moons[mask, 0], y=X_moons[mask, 1],
                             mode='markers', name=f'군집 {cluster}',
                             marker=dict(size=8)), row=1, col=1)

# DBSCAN
unique_labels = set(labels_dbscan_moons)
for cluster in unique_labels:
    mask = labels_dbscan_moons == cluster
    name = '노이즈' if cluster == -1 else f'군집 {cluster}'
    color = 'gray' if cluster == -1 else None
    fig.add_trace(go.Scatter(x=X_moons[mask, 0], y=X_moons[mask, 1],
                             mode='markers', name=name,
                             marker=dict(size=8, color=color)), row=1, col=2)

fig.update_layout(title='K-Means vs DBSCAN: 비선형 데이터', height=400)
fig.show()

In [ ]:
# 고객 데이터에 DBSCAN 적용
dbscan = DBSCAN(eps=0.5, min_samples=10)
labels_dbscan = dbscan.fit_predict(X_scaled)

# 결과 분석
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_noise = (labels_dbscan == -1).sum()

print(f"발견된 군집 수: {n_clusters}")
print(f"노이즈(이상치) 수: {n_noise}")
print(f"\n군집별 샘플 수:")
for label in sorted(set(labels_dbscan)):
    count = (labels_dbscan == label).sum()
    name = '노이즈' if label == -1 else f'군집 {label}'
    print(f"  {name}: {count}")

In [ ]:
# DBSCAN 결과 시각화
df_dbscan = df_customers.copy()
df_dbscan['Cluster'] = ['노이즈' if l == -1 else f'군집 {l}' for l in labels_dbscan]

fig = px.scatter(df_dbscan, x='Annual_Income', y='Spending_Score',
                 color='Cluster',
                 title=f'DBSCAN 결과 (eps=0.5, min_samples=10) - {n_clusters}개 군집, {n_noise}개 노이즈',
                 labels={'Annual_Income': '연간 소득', 'Spending_Score': '소비 점수'},
                 opacity=0.7)
fig.show()

### 💡 실무 예시: DBSCAN 활용

- **이상 탐지**: 노이즈(-1)로 분류된 고객 = 특이 패턴
- **지리 데이터**: 위치 기반 핫스팟 탐지
- **eps 튜닝**: k-distance plot으로 최적값 탐색

---

## 2.3 GMM (Gaussian Mixture Model)

GMM은 데이터가 여러 가우시안 분포의 혼합이라고 가정합니다.

**특징**:
- **확률적 할당**: 각 군집에 속할 확률 제공
- **소프트 클러스터링**: 경계가 명확하지 않은 경우 유용
- **EM 알고리즘**: Expectation-Maximization으로 학습

In [ ]:
# GMM 적용
gmm = GaussianMixture(n_components=4, random_state=42)
gmm.fit(X_scaled)

labels_gmm = gmm.predict(X_scaled)
probs_gmm = gmm.predict_proba(X_scaled)  # 각 군집에 속할 확률

print("첫 5개 샘플의 군집 소속 확률:")
prob_df = pd.DataFrame(probs_gmm[:5], columns=[f'군집 {i}' for i in range(4)])
prob_df['할당된 군집'] = labels_gmm[:5]
prob_df

In [ ]:
# GMM 결과 시각화
df_gmm = df_customers.copy()
df_gmm['Cluster'] = labels_gmm.astype(str)
df_gmm['Max_Prob'] = probs_gmm.max(axis=1)  # 가장 높은 확률

fig = px.scatter(df_gmm, x='Annual_Income', y='Spending_Score',
                 color='Cluster', size='Max_Prob',
                 title='GMM 결과: 크기 = 소속 확률',
                 labels={'Annual_Income': '연간 소득', 'Spending_Score': '소비 점수'},
                 color_discrete_sequence=px.colors.qualitative.Set1,
                 opacity=0.7)
fig.show()

print(f"\nGMM Silhouette Score: {silhouette_score(X_scaled, labels_gmm):.4f}")

In [ ]:
# BIC/AIC로 최적 군집 수 찾기
n_components_range = range(2, 10)
bic_scores = []
aic_scores = []

for n in n_components_range:
    gmm_temp = GaussianMixture(n_components=n, random_state=42)
    gmm_temp.fit(X_scaled)
    bic_scores.append(gmm_temp.bic(X_scaled))
    aic_scores.append(gmm_temp.aic(X_scaled))

# BIC/AIC Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(n_components_range), y=bic_scores,
                         mode='lines+markers', name='BIC'))
fig.add_trace(go.Scatter(x=list(n_components_range), y=aic_scores,
                         mode='lines+markers', name='AIC'))

fig.update_layout(title='GMM: BIC/AIC로 최적 군집 수 찾기 (낮을수록 좋음)',
                  xaxis_title='군집 수',
                  yaxis_title='Score')
fig.show()

print(f"BIC 최적 K: {list(n_components_range)[np.argmin(bic_scores)]}")
print(f"AIC 최적 K: {list(n_components_range)[np.argmin(aic_scores)]}")

### 💡 실무 예시: GMM 활용

- **불확실한 경계**: "이 고객은 70% 프리미엄, 30% 절약형"
- **마케팅 캠페인**: 확률 기반 타겟팅
- **이상 탐지**: 낮은 확률 = 어느 군집에도 맞지 않음

---

## 2.4 알고리즘 비교 종합

In [ ]:
# 모든 알고리즘 결과 종합 비교
algorithms = {
    'K-Means': labels_4,
    'Hierarchical': labels_hier,
    'DBSCAN': labels_dbscan,
    'GMM': labels_gmm
}

comparison_results = []
for name, labels in algorithms.items():
    # DBSCAN 노이즈 제외하고 평가
    if -1 in labels:
        mask = labels != -1
        if mask.sum() > 1:
            score = silhouette_score(X_scaled[mask], labels[mask])
        else:
            score = 0
        n_clusters = len(set(labels)) - 1
    else:
        score = silhouette_score(X_scaled, labels)
        n_clusters = len(set(labels))
    
    comparison_results.append({
        '알고리즘': name,
        '군집 수': n_clusters,
        'Silhouette Score': round(score, 4),
        '노이즈 수': (labels == -1).sum() if -1 in labels else 0
    })

comparison_df = pd.DataFrame(comparison_results)
print("=== 군집화 알고리즘 비교 ===")
comparison_df

In [ ]:
# 4개 알고리즘 결과 시각화
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=['K-Means', 'Hierarchical', 'DBSCAN', 'GMM'])

all_labels = [labels_4, labels_hier, labels_dbscan, labels_gmm]
positions = [(1,1), (1,2), (2,1), (2,2)]

for labels, (row, col) in zip(all_labels, positions):
    unique_labels = sorted(set(labels))
    for cluster in unique_labels:
        mask = labels == cluster
        name = '노이즈' if cluster == -1 else f'C{cluster}'
        fig.add_trace(go.Scatter(
            x=df_customers.loc[mask, 'Annual_Income'],
            y=df_customers.loc[mask, 'Spending_Score'],
            mode='markers', name=name,
            marker=dict(size=6, opacity=0.6),
            showlegend=(row==1 and col==1)
        ), row=row, col=col)

fig.update_layout(title='군집화 알고리즘 비교', height=700)
fig.show()

---

## 2.5 고객 세분화 실무 프로젝트

In [ ]:
# 최종 군집화 (K-Means K=4 선택)
final_model = KMeans(n_clusters=4, random_state=42, n_init=10)
df_customers['Segment'] = final_model.fit_predict(X_scaled)

# 세그먼트별 프로파일링
segment_profile = df_customers.groupby('Segment').agg({
    'Annual_Income': ['mean', 'std'],
    'Spending_Score': ['mean', 'std'],
    'Age': 'mean',
    'Visit_Frequency': 'mean',
    'CustomerID': 'count'
}).round(1)

segment_profile.columns = ['평균소득', '소득_std', '평균소비', '소비_std', 
                           '평균나이', '평균방문', '고객수']

# 세그먼트 이름 부여
segment_names = {
    0: '예산 의식형 (Budget Conscious)',
    1: '충동 소비형 (Impulse Shopper)',
    2: '절약 투자형 (Saver)',
    3: '프리미엄 VIP (Premium VIP)'
}

# 실제 데이터 기반으로 이름 재할당
for idx, row in segment_profile.iterrows():
    income = '고' if row['평균소득'] > 50 else '저'
    spending = '고' if row['평균소비'] > 50 else '저'
    if income == '저' and spending == '저':
        segment_names[idx] = '예산 의식형'
    elif income == '저' and spending == '고':
        segment_names[idx] = '충동 소비형'
    elif income == '고' and spending == '저':
        segment_names[idx] = '절약 투자형'
    else:
        segment_names[idx] = '프리미엄 VIP'

segment_profile['세그먼트명'] = segment_profile.index.map(segment_names)
print("=== 고객 세그먼트 프로파일 ===")
segment_profile

In [ ]:
# 세그먼트별 마케팅 전략
marketing_strategy = {
    '예산 의식형': '가성비 상품, 할인 쿠폰, 포인트 적립 강조',
    '충동 소비형': '한정판, 긴급 세일, 소셜 미디어 타겟팅',
    '절약 투자형': '프리미엄 멤버십, 투자 상품 연계, 장기 혜택',
    '프리미엄 VIP': 'VIP 전용 서비스, 럭셔리 라인, 개인화 추천'
}

print("=== 세그먼트별 마케팅 전략 ===")
for segment, strategy in marketing_strategy.items():
    print(f"\n[{segment}]")
    print(f"  전략: {strategy}")

In [ ]:
# 인터랙티브 군집 대시보드
df_customers['Segment_Name'] = df_customers['Segment'].map(segment_names)

fig = px.scatter(df_customers, x='Annual_Income', y='Spending_Score',
                 color='Segment_Name', size='Visit_Frequency',
                 hover_data=['CustomerID', 'Age'],
                 title='고객 세분화 대시보드',
                 labels={'Annual_Income': '연간 소득 (천만원)',
                        'Spending_Score': '소비 점수',
                        'Segment_Name': '세그먼트'},
                 color_discrete_sequence=px.colors.qualitative.Bold)

# 중심점 추가
centers_orig = scaler.inverse_transform(final_model.cluster_centers_)
for i, (cx, cy) in enumerate(centers_orig):
    fig.add_annotation(x=cx, y=cy, text=f"Center {i}",
                       showarrow=True, arrowhead=2)

fig.update_layout(height=600)
fig.show()

In [ ]:
# 세그먼트 비율 파이 차트
segment_counts = df_customers['Segment_Name'].value_counts()

fig = px.pie(values=segment_counts.values, names=segment_counts.index,
             title='고객 세그먼트 비율',
             color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. K-Means 기본 ⭐

**문제**: K=3으로 K-Means를 수행하고 각 군집의 샘플 수를 출력하세요.

```python
from sklearn.cluster import KMeans
# X_scaled는 이미 준비됨
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. Inertia 계산 ⭐

**문제**: K=5인 K-Means의 inertia 값을 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. Silhouette Score ⭐⭐

**문제**: K=4인 K-Means의 Silhouette Score를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. Elbow Plot 그리기 ⭐⭐

**문제**: K=1~10에 대해 inertia를 계산하고 px.line()으로 Elbow Plot을 그리세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 군집 시각화 ⭐⭐

**문제**: K=4 군집 결과를 px.scatter()로 시각화하세요. (color='Cluster')

In [ ]:
# 여기에 코드를 작성하세요


### Q6. DBSCAN 적용 ⭐⭐⭐

**문제**: eps=0.3, min_samples=5로 DBSCAN을 수행하고 군집 수와 노이즈 수를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 계층적 군집화 ⭐⭐⭐

**문제**: Ward linkage로 4개 군집의 계층적 군집화를 수행하고 Silhouette Score를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. GMM 확률 분석 ⭐⭐⭐⭐

**문제**: 4개 컴포넌트의 GMM을 학습하고, 가장 불확실한 샘플 5개(최대 확률이 가장 낮은)를 찾으세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 최적 K 탐색 ⭐⭐⭐⭐

**문제**: K=2~8에 대해 Silhouette Score를 계산하고, 최적 K를 찾아 시각화하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 고객 세분화 종합 ⭐⭐⭐⭐⭐

**문제**: 
1. K-Means (K=4)로 고객을 세분화하고
2. 각 세그먼트의 평균 소득, 소비점수, 고객 수를 DataFrame으로 정리하고
3. px.scatter()로 세그먼트별 시각화를 완성하세요.

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: K-Means 핵심 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| K-Means | 중심점 기반 군집화 | K를 미리 지정, 빠른 속도 |
| Elbow Method | inertia로 최적 K 탐색 | 꺾이는 지점 = 최적 K |
| Silhouette Score | 군집 품질 평가 (-1~1) | 1에 가까울수록 좋음 |
| 스케일링 | StandardScaler 필수 | 거리 기반 알고리즘 특성 |

### Part 2: 다양한 군집화 알고리즘 요약

| 알고리즘 | 특징 | 언제 사용? |
|----------|------|------------|
| 계층적 군집화 | Dendrogram, 계층 구조 | 군집 간 관계 파악, 소규모 데이터 |
| DBSCAN | 밀도 기반, 노이즈 탐지 | 이상치 탐지, 임의 모양 군집 |
| GMM | 확률적 할당 | 소프트 클러스터링, 불확실성 표현 |

### 💡 실무 팁

1. **알고리즘 선택 기준**:
   - 빠른 처리 필요 → K-Means
   - 이상치 탐지 필요 → DBSCAN
   - 계층 구조 파악 → Hierarchical
   - 확률 기반 분석 → GMM

2. **평가 방법**:
   - Silhouette Score: 모든 알고리즘에 적용 가능
   - Elbow Method: K-Means 전용
   - BIC/AIC: GMM 전용

3. **전처리 필수**:
   - StandardScaler: 거리 기반 알고리즘
   - 이상치 처리: 결과에 큰 영향

4. **비즈니스 활용**:
   - 고객 세분화 → 타겟 마케팅
   - 이상 탐지 → 사기 감지
   - 패턴 발견 → 추천 시스템